In [1]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
import re
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('D:/data_analysis_job.csv',encoding='gbk')
data.head()

,城市,招聘岗位,薪资,工作经验,学历,所招人数,专业要求,职位信息,公司类型,公司规模,所属行业
0,广州,年薪14万店铺销售数据分析,10-15万/年,1年经验,本科,若干,NaN,工作职责：1、门店运营及活动数据汇总分析，制作数据报表，做好预警监控2、负责每月管理层、分公...,民营公司,500-1000人,互联网/电子商务
1,广州,数据分析师,1.8-3万/月,2年经验,本科,若干,NaN,1、及时响应业务团队数据统计分析需求，输出可读性强的分析报告；2、结合分析结果及对业务需求的...,民营公司,NaN,计算机软件
2,广州,商品数据分析,7-8千/月,1年经验,NaN,1,NaN,1、商品相关数据进行采集统计分析，包括进销存分析，销存对比分析，同比、环比分析等，并将分析结...,民营公司,500-1000人,服装/纺织/皮革\n ...
3,广州,数据分析专员/助理,5-6千/月,2年经验,大专,1,NaN,岗位职责：1、负责进行客户资源电话筛选、售后电话回访工作2、负责维护客户关系，分析新老客户的...,民营公司,150-500人,新能源\n 汽车
4,广州,数据分析师,1-2万/月,2年经验,大专,2,NaN,根据电商发展战略，制定数据分析报表，进行行业、竞品分析，自品牌销售预测以及制定各类模型，为销...,民营公司,50-150人,互联网/电子商务


In [3]:
## 获取仅有一个数据的行记录的index
drop_index = []
for i in range(len(data)):
    if data.iloc[i].count() == 1:  ##判断依据
        drop_index.append(i)

data.drop(index=drop_index,axis=0,inplace=True)
data.reset_index(drop=True,inplace=True)

In [4]:
data.isna().sum()

城市          0
招聘岗位        0
薪资         80
工作经验      184
学历        196
所招人数        8
专业要求    10661
职位信息        0
公司类型        7
公司规模      624
所属行业        0
dtype: int64

In [5]:
## 对数据集进行复制，用于数据清理后与原数据进行对比
data1 = data.copy()

In [6]:
## 对数据缺失值进行填充

In [7]:
# 用该城市的平均薪资进行填充，单位统一使用 '万/年'作为统一的单位
def clean_salary(tag):
    if str(type(tag)) == "<class 'str'>":
        unit = tag[-3:]     ##原数据单位
        salary = tag[:-3]   ##原数据薪资
        if unit == '万/年':
            salary1 = salary
        elif unit == '万/月':
            x1 = round(float(salary.split('-')[0])*12,3)
            x2 = round(float(salary.split('-')[1])*12,3)
            salary1 = str(x1) + '-' + str(x2)
        elif unit == '千/月':
            x1 = round(float(salary.split('-')[0])/10*12,3)
            x2 = round(float(salary.split('-')[1])/10*12,3)
            salary1 = str(x1) + '-' + str(x2)
        elif unit == '/小时':   ###计算思路：一天工作8小时，一个月工作28days
            salary1 = str(round(float(salary[:-1])*8*28*12/10000,3))
        elif unit == '元/天':   ###计算思路：一个月工作28天
            salary1 = str(float(re.findall('\d+',tag)[0])*28*12/10000)
        else:
            if '年' and '万' in tag:
                salary1 = re.findall('\d+',tag)[0]
            else:
                salary1 = float(re.findall('\d+\.\d+',tag)[0])*12/10
        return salary1
data1['薪资'] = data1['薪资'].map(lambda x:clean_salary(x))
data1.rename(columns={'薪资':'薪资(万/年)'},inplace=True)

In [8]:
# 计算平均薪资
def average_salary(tag):
    if str(type(tag)) == "<class 'str'>":
        if '-' in tag:
            x1,x2 = float(tag.split('-')[0]),float(tag.split('-')[1])
            avg = (x1+x2)/2
        else:
            avg  = float(tag)
    else:
        avg = tag
    return avg
data1['平均薪资(万/年)'] = data1['薪资(万/年)'].map(lambda x:average_salary(x))

In [9]:
# 依据城市的不同进行填充
tmp = data1.groupby('城市')['平均薪资(万/年)'].mean().reset_index()
tmp.rename(columns={"平均薪资(万/年)":"mean"},inplace=True)
tmp['mean'] = tmp['mean'].map(lambda x:round(x,1))
data1 = data1.merge(tmp,on='城市',how='outer')
data1['薪资(万/年)'] = data1['薪资(万/年)'].fillna(data1['mean'])
data1['平均薪资(万/年)'] = data1['平均薪资(万/年)'].fillna(data1['mean'])

In [10]:
data2 = pd.DataFrame()
for i in data1['城市'].unique():
    tmp = data1[data1['城市'] == i]
    tmp['薪资等级'] = pd.cut(tmp['平均薪资(万/年)'],5,labels=["低","较低","中","较高","高"],right=False)
    data2 = pd.concat([tmp,data2])

data2.drop(['平均薪资(万/年)','mean'],axis=1,inplace=True)
del data1

In [11]:
# 除去使用城市作为填充依据外，也应加入薪资方面的信息作为填充条件(在传统认知了，工作经验越多，薪资水平越高)
experience_dict = {"无需经验":0,"1年经验":1,"2年经验":2,"3-4年经验":3,"5-7年经验":4,"8-9年经验":5,"10年以上经验":6}
data2['工作经验'] = data2['工作经验'].map(experience_dict)


In [12]:
# 计算平均工作经验，计算依据为城市和薪资等级
tmp = data2.groupby(['城市','薪资等级'])['工作经验'].mean().reset_index()
tmp['工作经验'] = tmp['工作经验'].map(lambda x:round(x,0))

In [13]:
# 使用使用平均工作经验进行填充
data2 = data2.merge(tmp,on=['城市','薪资等级'],how='outer')
data2['工作经验_x'] = data2['工作经验_x'].fillna(data2['工作经验_y'])
data2.drop('工作经验_y',axis=1,inplace=True)
data2.rename(columns={'工作经验_x':'工作经验'},inplace=True)

In [14]:
##键值对互换
experience_dict_T = {}
for k,v in experience_dict.items():
    experience_dict_T[v] = k
##重新映射为
data2['工作经验'] = data2['工作经验'].map(experience_dict_T)
del tmp

In [15]:
# 与工作经验类似处理
education_dict = {"中专":0,"高中":1,"大专":2,"本科":3,"硕士":4,"博士":5}
data2['学历'] = data2['学历'].map(education_dict)

In [16]:
tmp = data2.groupby(['城市','薪资等级'])['学历'].mean().reset_index()
tmp['学历'] = tmp['学历'].map(lambda x:round(x,0))

In [17]:
data2 = data2.merge(tmp,on=['城市','薪资等级'],how='outer')
data2['学历_x'] = data2['学历_x'].fillna(data2['学历_y'])
data2.drop('学历_y',axis=1,inplace=True)
data2.rename(columns={'学历_x':'学历'},inplace=True)

In [18]:
##键值对互换
education_dict_T = {}
for k,v in education_dict.items():
    education_dict_T[v] = k
##重新映射为
data2['学历'] = data2['学历'].map(education_dict_T)
del tmp

In [19]:
# 所招人数用若干填充
data2['所招人数'] = data2['所招人数'].fillna('若干')

In [20]:
# 专业要求缺失值较多，直接删除该列
len(data2[data2['专业要求'].isna()==True])/len(data2)

0.8642625607779578

In [21]:
data2.drop('专业要求',axis=1,inplace=True)

In [22]:
# 使用"众数"作为填充依据
fill_str = data2['公司类型'].value_counts().idxmax()
data2['公司类型'] = data2['公司类型'].fillna(fill_str)

In [23]:
# 和薪资列处理方法类似，同时考虑城市和公司类型的情况
def avg_people(tag):
    if str(type(tag)) == "<class 'str'>":
        if '-' not in tag:
            r = float(re.findall('\d+',tag)[0])
        else:
            people = tag[:-1]
            x1 = float(people.split('-')[0])
            x2 = float(people.split('-')[1])
            r = (x1+x2)/2
    else:
        r = tag
    return r

data2['平均公司规模'] = data2['公司规模'].map(lambda x:avg_people(x))

In [24]:
tmp = data2.groupby(['城市','公司类型'])['平均公司规模'].mean().reset_index()
tmp.rename(columns={"平均公司规模":"mean"},inplace=True)
tmp['mean'] = tmp['mean'].map(lambda x:round(x,0))
data2 = data2.merge(tmp,on=['城市','公司类型'],how='outer')
data2['公司规模'] = data2['公司规模'].fillna(data2['mean'])
data2.drop(['平均公司规模','mean'],axis=1,inplace=True)
del tmp

In [25]:
## 保存处理好的数据
data2.to_csv('D:/data_analysis_job_clean.csv',encoding='utf-8-sig',index=False)

In [26]:
## ------------可视化--------------------
## ------------可视化--------------------
## ------------可视化--------------------
## ------------可视化--------------------

In [27]:
import pandas as pd

# 加载CSV文件
df = pd.read_csv('D:/data_analysis_job_clean.csv')

# 查看数据的前几行，以了解数据结构
df.head()

,城市,招聘岗位,薪资(万/年),工作经验,学历,所招人数,职位信息,公司类型,公司规模,所属行业,薪资等级
0,上海,销售数据分析员-汉高外包项目,8.4-10.8,1年经验,本科,1,1.全国销售报表（业绩报表，绩效核算数据）的统计发送；2.销售部重点项目协助跟进；3.销售部...,上市公司,3897.0,快速消费品(食品、饮料、化妆品)\n ...,低
1,上海,数据分析专员,8.4-10.8,1年经验,大专,1,职位描述1、对日常业务数据进行分析，定期根据数据分析制作报表，充分了解数据管理在销售工作中的...,上市公司,500-1000人,服装/纺织/皮革\n ...,低
2,上海,数据分析师（会员）(J11928),9.6-13.2,2年经验,本科,1,工作职责:1、根据业务部门需求及时提取并编制相关数据报表，包括数据清洗、数据分析及数据仓库管...,上市公司,1000-5000人,快速消费品(食品、饮料、化妆品),低
3,上海,数据分析员,6.0-12.0,无需经验,本科,1,根据业务需求，制作常规周期性报告和定制化报告，为管理层提供数据支持日常业务数据的监控、诊断和...,上市公司,1000-5000人,制药/生物工程,低
4,上海,Python数据分析师,8.4-16.8,3-4年经验,本科,1,工作职责：1、根据给出的优化目标，以及数据结构和参数，从业务的角度考虑问题，提出分析和解决的...,上市公司,150-500人,计算机软件\n 互联网...,低


In [28]:
# 基本统计描述
print(df.describe())

# 查看每列的数据类型和非空值的数量
print(df.info())

           城市   招聘岗位    薪资(万/年)    工作经验     学历   所招人数  \
count   12340  12336      12336   12336  12336  12340   
unique     12   6664        429       7      6     24   
top        南京  数据分析师  12.0-18.0  3-4年经验     本科      1   
freq     2454    687       1328    3772   6096   6567   

                                                     职位信息   公司类型      公司规模  \
count                                               12336  12340     12340   
unique                                              10365     10        47   
top     研发人员（生物信息学 /?宏基因组学）岗位职责1.负责与精准医学相关的宏基因组、转录组和其他...   民营公司  150-500人   
freq                                                   11   8241      2899   

         所属行业   薪资等级  
count   12336  12340  
unique    955      5  
top     计算机软件      低  
freq      731  10011  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12340 entries, 0 to 12339
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   城市       12340 non-n

In [29]:
import pandas as pd
from pyecharts.charts import Pie
from pyecharts.charts import Bar
from pyecharts import options as opts
from pyecharts.globals import ThemeType, CurrentConfig, OnlineHostType, NotebookType

# 统计学历要求
education_counts = df['学历'].value_counts()

# 创建一个饼图
pie = Pie(init_opts=opts.InitOpts(width="900px", height="600px"))

# 添加数据
pie.add(
    series_name="学历要求分布",
    data_pair=[list(z) for z in zip(education_counts.index.tolist(), education_counts.values.tolist())],
    radius=["30%", "75%"],
    label_opts=opts.LabelOpts(formatter="{b}: {c} ({d}%)", position="outside")
)

# 设置全局配置项
pie.set_global_opts(
    title_opts=opts.TitleOpts(title="学历要求分布", pos_left="center"),
    legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%")
)

# 设置系列配置和视觉效果
pie.set_series_opts(
    tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"),
    label_opts=opts.LabelOpts(is_show=True, position="outside", formatter="{b}: {d}%")
)

# 渲染图表到Notebook
pie.render_notebook()


In [30]:
# 统计工作经验
experience_counts = df['工作经验'].value_counts()

# 创建柱状图对象
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))

# 添加数据，确保数据是列表格式
bar.add_xaxis(experience_counts.index.tolist())
bar.add_yaxis("岗位数量", experience_counts.values.tolist())

# 设置全局配置
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="岗位对工作经验的要求", pos_left="center"),
    legend_opts=opts.LegendOpts(is_show=False),  # 不显示图例
)

# 设置系列配置和视觉效果
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True, position="top"),  # 显示数据标签
)

# 在Jupyter Notebook中渲染图表
bar.render_notebook()

In [31]:
# 统计城市
city_counts = df['城市'].value_counts()

# 创建柱状图对象
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))

# 添加数据，确保数据是列表格式
bar.add_xaxis(city_counts.index.tolist())
bar.add_yaxis("岗位数量", city_counts.values.tolist())

# 设置全局配置
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="各城市岗位数量情况", pos_left="center"),
    legend_opts=opts.LegendOpts(is_show=False),  # 不显示图例
)

# 设置系列配置和视觉效果
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True, position="top"),  # 显示数据标签
)

# 在Jupyter Notebook中渲染图表
bar.render_notebook()

In [32]:
# 定义一个函数来解析薪资字段，只取区间的最低值
def parse_salary(salary_str):
    try:
        parts = salary_str.split('-')
        return float(parts[0])  # 只取区间的最低值
    except:
        return np.nan  # 如果转换失败，返回NaN

df['薪资(万/年)'] = df['薪资(万/年)'].apply(parse_salary)

# 计算每个城市的平均薪资
city_avg_salary = df.groupby('城市')['薪资(万/年)'].mean()

# 创建柱状图对象
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))

# 添加数据，确保数据是列表格式
bar.add_xaxis(city_avg_salary.index.tolist())
bar.add_yaxis("平均薪资(万/年)", city_avg_salary.values.round(2).tolist())

# 设置全局配置
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="各城市岗位平均薪资情况", pos_left="center"),
    legend_opts=opts.LegendOpts(is_show=False),  # 不显示图例
)

# 设置系列配置和视觉效果
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True, position="top"),  # 显示数据标签
)

# 在Jupyter Notebook中渲染图表
bar.render_notebook()

In [33]:

# 计算每个城市的平均薪资
education_avg_salary = df.groupby('学历')['薪资(万/年)'].mean()

# 创建柱状图对象
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))

# 添加数据，确保数据是列表格式
bar.add_xaxis(education_avg_salary.index.tolist())
bar.add_yaxis("平均薪资(万/年)", education_avg_salary.values.round(2).tolist())

# 设置全局配置
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="各学历对应平均薪资情况", pos_left="center"),
    legend_opts=opts.LegendOpts(is_show=False),  # 不显示图例
)

# 设置系列配置和视觉效果
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True, position="top"),  # 显示数据标签
)

# 在Jupyter Notebook中渲染图表
bar.render_notebook()

In [34]:
# 统计招聘岗位
job_counts = df['招聘岗位'].value_counts()

# 取前5个最常见的岗位
top_jobs = job_counts.head(5)

# 创建柱状图对象
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))

# 添加数据，确保数据是列表格式
bar.add_xaxis(top_jobs.index.tolist())
bar.add_yaxis("岗位数量", top_jobs.values.tolist())

# 设置全局配置
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="数量最多的5个招聘岗位", pos_left="center"),
    legend_opts=opts.LegendOpts(is_show=False),  # 不显示图例
)

# 设置系列配置和视觉效果
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True, position="top"),  # 显示数据标签
)

# 在Jupyter Notebook中渲染图表
bar.render_notebook()

In [35]:
# 统计工作经验
company_types = df['公司类型'].value_counts()

# 创建柱状图对象
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))

# 添加 X 轴数据（公司类型）
bar.add_xaxis(company_types.index.tolist())

# 添加 Y 轴数据，系列名称为"公司类型"
bar.add_yaxis("公司类型", company_types.values.tolist())

# 设置全局配置
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="招聘公司类型情况", pos_left="center"),
    legend_opts=opts.LegendOpts(is_show=False),  # 不显示图例
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)), # 如果标签太长可以考虑旋转标签
)

# 设置系列配置和视觉效果
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True, position="top")  # 在柱状图顶部显示数据
)

# 在 Jupyter Notebook 中渲染图表
bar.render_notebook()

In [36]:
# 统计公司规模
company_scale = df['公司规模'].value_counts()

# 只显示前N个类别，其他合并为“其他”
N = 5  # 可以根据需要调整显示的类别数量
top_company_scale = company_scale[:N]  # 取前N个最常见的

# 找出并分别统计10000人以上和5000-10000人
over_10000 = company_scale[company_scale.index.str.contains('10000人以上')]  # 假设描述为 "1000-4999人"
between_5000_10000 = company_scale[company_scale.index.str.contains('5000-10000')]  # 假设描述为 "5000-10000人"

# 合并其他未包括的规模
top_company_scale['10000人以上'] = over_10000.sum()
top_company_scale['5000-10000人'] = between_5000_10000.sum()

# 创建柱状图对象
bar = Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))

# 添加 X 轴数据（公司类型）
bar.add_xaxis(top_company_scale.index.tolist())

# 添加 Y 轴数据，系列名称为"公司类型"
bar.add_yaxis("公司规模", top_company_scale.values.tolist())

# 设置全局配置
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="招聘公司规模情况", pos_left="center"),
    legend_opts=opts.LegendOpts(is_show=False),  # 不显示图例
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)), # 如果标签太长可以考虑旋转标签
)

# 设置系列配置和视觉效果
bar.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True, position="top")  # 在柱状图顶部显示数据
)

# 在 Jupyter Notebook 中渲染图表
bar.render_notebook()
